## Policy Gradient

#### [CSCI-UA.0473 Spring 2020] Introduction to Machine Learning

#### May 06 2020
Prepared by Sean Welleck

In [ ]:
%pylab inline

import time
import gym
from collections import defaultdict
from itertools import chain
import random
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import scipy.signal
import random
import pickle

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

## Reinforcement Learning Preliminaries

Goal: find a **policy** $\pi_{\theta}(a|s)$ that maximizes the **expected total reward** of its **trajectories**.

\begin{align*}
s\in \mathcal{S} & \quad\text{'state' in 'state space'}\\
a\in \mathcal{A} & \quad\text{'action' in 'action space'}\\
r\in \mathbb{R}  & \quad\text{'reward'}\\
\tau=(s_0, a_0, r_0, s_1, a_1, r_1,\ldots, s_T) & \quad\text{trajectory (or 'episode')}\\
R(\tau) =\sum_{t=0}^{T} r_t         & \quad\text{'total reward' (or 'return')}
\end{align*}

\begin{align*}
    \color{blue}{p(\tau)} &= \underbrace{p(s_0)}_{\text{initial state distribution}}\prod_{t=0}^{T-1} \underbrace{\pi_{\theta}(a_t|s_t)}_{\text{policy}}\underbrace{p(s_{t+1}|a_t, s_t)}_{\text{transition distribution}}
\end{align*}

\begin{align*}
    \displaystyle J(\theta) &= \mathop{\mathbb{E}}_{\tau\sim \color{blue}{p(\tau)}}\left[R(\tau)\right]
\end{align*}

#### Policy Gradient

Sample $K$ trajectories, then form the gradient estimator

\begin{align}
    \nabla_{\theta} &\approx \frac{1}{K}\sum_{i=1}^{K} \sum_{t=1}^{|\tau|} \underbrace{\left(R_t - b(s_t^{(i)})\right)}_{\text{"advantage" A(s,a)}}\nabla_{\theta}\log \pi_{\theta}(a^{(i)}_t|s_t^{(i)})
\end{align}

$R_t$ can be the full return $R(\tau)$ or $\sum_{t'=t}^{|\tau|}r_t$ (we will see specifics below).

### Warmup: One-step Toy Example

**State**: $\emptyset$ (single state)

**Action**: $\{1,2,\ldots,A\}$

**Reward**: 
\begin{align}
r(s,a) &= \begin{cases}
    1.0 & a=1\\
    0.9 & a\in \{2,\ldots,\frac{A}{2}\}\\
    0.0 & \text{ otherwise}
\end{cases}
\end{align}

The policy must learn to choose $a = 1$ to maximize the reward ($1.0$).

We will use **policy gradient**, and see how the **baseline** and **number of samples** affect the results.

#### Reward (unknown to the policy)

In [ ]:
class Reward(object):
    def __init__(self, num_actions):
        rs = torch.zeros(num_actions)
        rs[0] = 1.0
        rs[1:int(num_actions//2)] = 0.90
        self.rs = rs

    def __call__(self, actions):
        return self.rs[actions]

#### Policy

The policy computes (the parameters of) a categorical distribution over actions, given a state.
We parameterize this mapping with a neural network:

\begin{align*}
    \pi_{\theta}(a|s) &= \text{softmax}(f_{\theta}(s))\\
    f_{\theta}(s)&=W_2(\text{tanh}(W_1s + b_1)) + b_2
\end{align*}

*a neural network isn't needed for this simple problem, but we'll use a similar model below for a more complex problem

In [ ]:
class Policy(nn.Module):
    def __init__(self, input_size, hidden_size, num_actions):
        super(Policy, self).__init__()
        self.w1 = nn.Linear(input_size, hidden_size)
        self.w2 = nn.Linear(hidden_size, num_actions)

    def forward(self, state):
        x = self.w1(state)
        x = F.tanh(x)
        log_ps = torch.log_softmax(self.w2(x), -1)
        return log_ps
    
    def sample(self, log_ps):
        actions = log_ps.exp().multinomial(1)
        return actions

#### Policy Gradient

\begin{align}
    \nabla_{\theta} &\approx \frac{1}{K}\sum_{i=1}^{K} \underbrace{\left(r(a^{(i)}) - b(s)\right)}_{\text{"advantage"} A(s,a)}\nabla_{\theta}\log \pi_{\theta}(a^{(i)}|s)
\end{align}

[NOTE:] To use the auto-differentiation library (PyTorch) in a standard way, we actually form a **loss function whose gradient is the policy gradient**.

In [ ]:
def loss(log_ps, actions, advantages):
    policy_grad_loss = -torch.mean(advantages*log_ps.gather(1, actions))
    return policy_grad_loss

#### Training Loop

In this initial case, we will use **no baseline**, i.e.

$$A(s, a) = r(a)$$

In [ ]:
stats = defaultdict(list)

In [ ]:
num_actions = 100
num_samples = 1

num_episodes = 20000

In [ ]:
policy = Policy(
    input_size=1, 
    hidden_size=8, 
    num_actions=num_actions
)
reward_func = Reward(num_actions)
optimizer = optim.Adam(policy.parameters(), lr=1e-3)

running_reward = 0
for i in range(num_episodes):
    state = torch.ones(num_samples, 1)
    log_ps = policy(state)
    actions = policy.sample(log_ps)
    rewards = reward_func(actions)

    optimizer.zero_grad()
    pg_loss = loss(log_ps, actions, rewards)
    pg_loss.backward()
    optimizer.step()

    for er in rewards:
        running_reward = er.item() if running_reward is None else (
            0.99 * running_reward + 0.01 * er.item())

    if (i+1) % 1000 == 0 or i == 0:
        print('Batch %d complete (episode %d), batch avg. reward: %.2f, running reward: %.3f' %
              (i+1, (i+1)*num_samples, torch.mean(rewards).item(), running_reward))
    
        stats[('none', num_samples)].append(running_reward)

#### Running-average baseline

Given a history of rewards from previous batches, $\{r_1,\ldots,r_N\}$, let $b_{n} = 0.1r_{n-1} + 0.9 b_{n-1}$ (with $b_0=0,r_0=0$).

$$A(s,a) = r(a) - b_n$$

In [ ]:
policy = Policy(
    input_size=1, 
    hidden_size=8, 
    num_actions=num_actions
)
reward_func = Reward(num_actions)
optimizer = optim.Adam(policy.parameters(), lr=1e-3)

running_reward = 0
for i in range(num_episodes):
    state = torch.ones(num_samples, 1)
    log_ps = policy(state)
    actions = policy.sample(log_ps)
    rewards = reward_func(actions)
    
    # ------ baseline
    advantages = torch.clamp(rewards - running_reward, min=0)
    # ------

    optimizer.zero_grad()
    pg_loss = loss(log_ps, actions, advantages)
    pg_loss.backward()
    optimizer.step()

    for er in rewards:
        running_reward = er.item() if running_reward is None else (
            0.99 * running_reward + 0.01 * er.item())

    if (i+1) % 1000 == 0 or i == 0:
        print('Batch %d complete (episode %d), batch avg. reward: %.2f, running reward: %.3f' %
              (i+1, (i+1)*num_samples, torch.mean(rewards).item(), running_reward))
    
        stats[('avg-baseline', num_samples)].append(running_reward)

In [ ]:
fig, axs = plt.subplots(1, figsize=(8, 5))

axs.plot(stats[('none', num_samples)])
axs.plot(stats[('avg-baseline', num_samples)])
axs.hlines(0.90, -0.5, len(stats[('none', num_samples)])-1, alpha=0.8, linestyle=':')
axs.hlines(1.0, -0.5, len(stats[('none', num_samples)])-1, alpha=0.8, linestyle=':')

axs.set_xlim([-0.2, len(stats[('none', num_samples)])-1])
axs.set_ylabel('reward')
axs.set_xlabel('number of batches')
axs.legend(['none', 'avg-baseline'], loc='lower right');

## Example: Atari Pong

In [ ]:
env = gym.make('PongDeterministic-v4')
print("action space: %s" % (env.action_space))

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20, 10))

state = env.reset()
print("state size: %s" % str(state.shape))

axs[0].imshow(state)
axs[0].axis('off')
axs[0].set_title('$s_0$', fontsize=20)

for i in range(4):
    action = 3 # up=2, down=3
    state2, reward, done, info = env.step(action)  
    
    axs[i+1].imshow(state2)
    axs[i+1].axis('off')
    axs[i+1].set_title('$s_%d$' % (i+1), fontsize=20)


#### Example: Episode with Random Policy

In order to rollout a full trajectory to get a better sense of the problem, we'll use a simple random policy:

In [ ]:
def random_policy(state):
    action = np.random.randint(6)
    return action

In [ ]:
state = env.reset()
done = False
rewards = []
while not done:
    action = random_policy(state)
    state, reward, done, info = env.step(action)
    rewards.append(reward)
    env.render()
    time.sleep(0.05)
    
env.close()

In [ ]:
rewards = np.array(rewards)
print("episode length: %d" % len(rewards))
print("number of games: %d" % (rewards != 0).sum())

In [ ]:
print("episode rewards:")
i = 0
for j, index in enumerate(rewards.nonzero()[0]):
    print("\tgame %d: %s" % (j, ','.join(map(str, rewards[i:index+1].astype(int).tolist()))))
    i = index+1
    
print("\nreturn: %.1f" % (rewards.sum()))

## Policy Gradient

Recall from above that the policy gradient estimator weights the gradient of the log-probability by an 'advantage' term, which we can write as,
\begin{align}
    \nabla_{\theta} \approx \frac{1}{K}\sum_{k=1}^K\sum_{t=1}^{|\tau^{(k)}|}A(s_t^{(k)}, a_t^{(k)})\nabla_{\theta}\log \pi_{\theta}(a_t^{(k)}|s_t^{(k)})
\end{align}
where $\tau^{(k)} \sim p(\tau)$, and $\tau^{(k)} = (s_0^{(k)}, a_0^{(k)}, r_0^{(k)},\ldots,s_T^{(k)}).$

#### Variance reduction - discounting

Notice that the trajectories above are long. Consider the cumulative return of the first step (assuming the policy loses every game),
\begin{align}
R(s_1,a_1)&=\sum_{i=1}^{|\tau|}r_t\\
          &=-21.
\end{align}

Intuitively, the first action has no effect on the reward $r_t$ for $t$ sufficiently far in the future, so we want to downweight rewards far in the future.
In practice this is done through **discounting**,
\begin{align}
    R(s_t,a_t) &= \sum_{t'=t}^{|\tau|} \gamma^{t'-t}r_{t'}
\end{align}

which effectively includes $\frac{1}{1-\gamma}$ timesteps in the sum.
This introduces bias, but generally reduces variance.

In [ ]:
def _discount(rewards, gamma):
    return scipy.signal.lfilter([1], [1, -gamma], rewards[::-1], axis=0)[::-1]

#### Variance reduction - learned baseline

Next, we will introduce a **learned baseline**, which is known as a **state-value function** $V_{\phi}(s_t)$, so that the advantage is,
\begin{align}
    A(s_t,a_t) &= R(s_t,a_t) - V_{\phi}(s_t).
\end{align}

The state-value function $V_{\phi}(s_t)$ will be trained to estimate the expected return at state $s_t$,
\begin{align}
    \displaystyle V_{\phi}(s_t) &= \mathop{\mathbb{E}}_{a\sim \pi_{\theta}}[R(s_t, a)]
\end{align}

Intuitively, the advantage is thus positive when the observed return is 'higher than expected'.

In [ ]:
class ValueFunction(nn.Module):
    def __init__(self, hidden_size):
        super(ValueFunction, self).__init__()
        self.w1 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = self.w1(state)
        return x

#### Losses

We train $V(s_t)$ with a regression loss ($L_2$ loss) using the observed returns:

In [ ]:
def vf_loss(values, returns):
    loss = 0.5*torch.mean((values - returns)**2)
    return loss

def pg_loss(log_ps, actions, advantages, K):
    loss = -torch.sum(log_ps.gather(1, actions) * advantages) / K
    return loss

def total_loss(log_ps, actions, values, returns, advantages, args):
    actions = torch.tensor(actions, device=log_ps.device)
    advantages = torch.tensor(advantages, device=log_ps.device)
    returns = torch.tensor(returns, device=log_ps.device)
    
    entropy = -torch.sum(log_ps.exp()*log_ps, 1).mean()

    loss = (
        pg_loss(log_ps, actions, advantages, args.num_envs) +
        args.vf_weight*vf_loss(values, returns) -
        args.entropy_weight*entropy
    )
    return loss

#### Policy Network

The policy now includes the value function; in practice, the value function's input is the hidden state computed by the policy.

In [ ]:
class Policy(nn.Module):
    def __init__(self, input_size, hidden_size, num_actions, gamma):
        super(Policy, self).__init__()
        self.w1 = nn.Linear(input_size, hidden_size)
        self.w2 = nn.Linear(hidden_size, num_actions)
        self.vf = ValueFunction(hidden_size)

    def forward(self, state):
        x = self.w1(state)
        x = F.relu(x)

        values = self.vf(x)
        log_ps = torch.log_softmax(self.w2(x), -1)
        return log_ps, values

    def act(self, log_ps):
        actions = log_ps.argmax(dim=1)
        return actions

    def sample(self, log_ps):
        actions = log_ps.exp().multinomial(1)
        return actions

#### Train episode

In [ ]:
def train_episode(policy, optimizer, envs, preprocessors, args):
    """Complete an episode's worth of training for each environment."""
    num_envs = len(envs)

    # Buffers to hold trajectories, e.g. `env_xs[i]` will hold the observations for environment `i`.
    env_xs, env_as = _2d_list(num_envs), _2d_list(num_envs)
    env_rs, env_vs = _2d_list(num_envs), _2d_list(num_envs)
    episode_rs = np.zeros(num_envs, dtype=np.float)

    for p in preprocessors:
        p.reset()
    observations = [p.preprocess(e.reset()) for p, e in zip(preprocessors, envs)]

    done = np.array([False for _ in range(num_envs)])
    all_done = False
    t = 1
    while not all_done:
        step_xs = np.vstack([o.ravel() for o in observations])

        # Get actions and values for all environments in a single forward pass.
        x = torch.tensor(step_xs, dtype=torch.float, device=args.device)
        step_logps, step_vs = policy.forward(x)
        step_as = policy.sample(step_logps)

        # Step each environment whose episode has not completed.
        for i, env in enumerate(envs):
            if not done[i]:
                obs, r, done[i], _ = env.step(step_as[i])

                # Record the observation, action, value, and reward in the buffers.
                env_xs[i].append(step_xs[i].ravel())
                env_as[i].append(step_as[i])
                env_vs[i].append(step_vs[i][0].item())
                env_rs[i].append(r)
                episode_rs[i] += r

                # Add 0 as the state value when done.
                if done[i]:
                    env_vs[i].append(0.0)
                else:
                    observations[i] = preprocessors[i].preprocess(obs)

        # Perform an update every `t_max` steps.
        if t == args.t_max:
            # If the episode has not finished, add current state's value. This will be used to
            # 'bootstrap' the final return (see Algorithm S3 in A3C paper).
            x = torch.tensor(np.vstack(observations), dtype=torch.float, device=args.device)
            _, extra_vs = policy.forward(x)
            for i in range(num_envs):
                if not done[i]:
                    env_vs[i].append(extra_vs[i][0].item())

            # Perform update and clear buffers.
            train_step(policy, optimizer, env_xs, env_as, env_rs, env_vs, args)
            env_xs, env_as = _2d_list(num_envs), _2d_list(num_envs)
            env_rs, env_vs = _2d_list(num_envs), _2d_list(num_envs)
            t = 0

        all_done = np.all(done)
        t += 1

    # Perform a final update when all episodes are finished.
    if len(env_xs[0]) > 0:
        train_step(policy, optimizer, env_xs, env_as, env_rs, env_vs, args)

    return episode_rs

def _2d_list(n):
    return [[] for _ in range(n)]

#### Train step

In [ ]:
def train_step(policy, optimizer, env_xs, env_as, env_rs, env_vs, args):
    # Stack all the observations and actions.
    xs = np.vstack(list(chain.from_iterable(env_xs)))
    as_ = np.array(list(chain.from_iterable(env_as)))[:, np.newaxis]

    # Compute discounted rewards and advantages.
    drs, advs = [], []
    gamma = args.gamma
    for i in range(len(env_vs)):
        # Compute discounted rewards with a 'bootstrapped' final value.
        rs_bootstrap = [] if env_rs[i] == [] else env_rs[i] + [env_vs[i][-1]]
        drs_ = _discount(rs_bootstrap, gamma)[:-1]
        drs.extend(drs_)
        advs.extend(drs_ - np.array(env_vs[i][:-1]))

    drs = np.array(drs)[:, np.newaxis]
    advs = np.array(advs)[:, np.newaxis]

    # Forward and backward pass now that we have everything computed
    optimizer.zero_grad()
    xs = torch.tensor(xs, dtype=torch.float, device=args.device)
    log_ps, vs = policy.forward(xs)
    loss_ = total_loss(log_ps, as_, vs, drs, advs, args)
    loss_.backward()

    torch.nn.utils.clip_grad_norm_(policy.parameters(), args.max_grad_norm)
    optimizer.step()

#### Utils

In [ ]:
class Atari8080Preprocessor(object):
    def __init__(self):
        self.prev = None
        self.obs_size = 80*80

    def reset(self):
        self.prev = None

    def preprocess(self, img):
        """ Preprocess a 210x160x3 uint8 frame into a 6400 (80x80) (1 x input_size) float vector."""
        # Crop, down-sample, erase background and set foreground to 1.
        # Ref: https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5
        img = img[35:195]
        img = img[::2, ::2, 0]
        img[img == 144] = 0
        img[img == 109] = 0
        img[img != 0] = 1
        curr = np.expand_dims(img.astype(np.float).ravel(), axis=0)
        # Subtract the last preprocessed image.
        diff = curr - self.prev if self.prev is not None else np.zeros((1, curr.shape[1]))
        self.prev = curr
        return diff

### Training 

We train the policy by repeatedly calling `train_episode`.
A full implementation with command-line arguments is in `train.py`.

It should take ~45 minutes on a macbook pro to solve the game (defined as a running reward of >20).

In [ ]:
args = {
    'seed': 0,
    'num_envs': 16,
    't_max': 50,
    'env_type': 'PongDeterministic-v4',
    'num_episodes': 1000000,
    'print_every': 1,
    'save_every': 5,
    'learning_rate': 1e-3,
    'gamma': 0.99,
    'entropy_weight': 0.01,
    'vf_weight': 2.0,
    'hidden_size': 200,
    'max_grad_norm': 1.0
}

class dotdict(dict):
    """ dot.notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict(args)
stats = defaultdict(list)

np.random.seed(args.seed)
torch.random.manual_seed(args.seed)
random.seed(args.seed)

# Create and seed the environments
envs = [gym.make(args.env_type) for _ in range(args.num_envs)]
preprocessors = [Atari8080Preprocessor() for _ in range(args.num_envs)]

input_size = preprocessors[0].obs_size
num_actions = envs[0].action_space.n
print("Input size %d\nNum actions %d" % (input_size, num_actions))

for i, env in enumerate(envs):
    env.seed(i+args.seed)

policy = Policy(input_size, args.hidden_size, num_actions, args.gamma)
policy.to(args.device)
optimizer = optim.Adam(policy.parameters(), lr=args.learning_rate)


# Train
running_reward = None
start = time.time()
for i in range(args.num_episodes):
    tic = time.time()
    episode_rewards = train_episode(policy, optimizer, envs, preprocessors, args)

    for er in episode_rewards:
        running_reward = er if running_reward is None else (
            0.99 * running_reward + 0.01 * er)

    if i % args.print_every == 0:
        print('Batch %d complete (%.2fs) (%.1fs elapsed) (episode %d), batch avg. reward: %.2f, running reward: %.3f' %
              (i, time.time()-tic, time.time() - start, (i+1)*args.num_envs, np.mean(episode_rewards), running_reward))
        stats['running_reward'].append(running_reward)
        
    if i % args.save_every == 0:
        torch.save({"state_dict": policy.state_dict()}, "model.pt")
        pickle.dump(stats, open("stats.pkl", "wb"))
        print("Model saved. (running reward %.2f)" % running_reward)
    
    if running_reward >= 20:
        break

### Run the Learned Policy

In [ ]:
import pickle
import os
load_dir = './checkpoint'
stats = pickle.load(open(os.path.join(load_dir, 'stats.pkl'), 'rb'))

policy = Policy(input_size, args.hidden_size, num_actions, args.gamma)
policy.load_state_dict(
    torch.load(os.path.join(load_dir, 'model.pt'))['state_dict']
)

In [ ]:
env = gym.make(args.env_type)
pre = Atari8080Preprocessor()

state = pre.preprocess(env.reset())
done = False
while not done:
    state = torch.tensor(state, dtype=torch.float)
    log_ps, _ = policy(state)
    action = policy.act(log_ps)[0].item()
    state, reward, done, info = env.step(action)
    state = pre.preprocess(state)
    
    env.render()
    time.sleep(0.05)
    
env.close()